# Python3 + Appium + Andriod 趣头条脚本

功能包括: 签到，金币领取，新闻阅读(仅限深圳地区)，小视频，视频

## 开发环境配置

[Selenium 安装](https://germey.gitbooks.io/python3webspider/content/1.2.2-Selenium%E7%9A%84%E5%AE%89%E8%A3%85.html)
[appnium 安装](https://germey.gitbooks.io/python3webspider/content/1.7.3-Appium%E7%9A%84%E5%AE%89%E8%A3%85.html)


### Appium基础

- platformName: 手机操作系统
- deviceName: 手机类型
将andriod手机链接电脑后，电脑打开Appium，同时手机打开USB调试功能，可通过 `adb devices -l` 命令来测试链接状态，如果出现类似结果，说明PC已经正确连接手机。

```
List of devices attached
177dca8c               device product:c7ltezc model:SM_C7000 device:c7ltechn transport_id:3
```

- newCommandTimeout: 两条appium命令间的最长时间间隔，若超过这个时间，appium会自动结束并退出app

- noReset, fullReset: noReset 不要在会话前重置应用状态。默认值false。 fullReset (Android) 通过卸载而不是清空数据来重置应用状态。在Android上, 这也会在会话结束后自动清除被测应用。默认值false。【建议用 noReset, 可以保存设置中的登录状态，这样滤过复杂的登录验证】

- appActivity, appPackage: appActivity与appPackage指用于启动待测app的activityName与packageName


### 趣头条

- 趣头条的元素大概3-4天就会更换一次，所以脚本中元素涉及 resource_id 处需要不定期修改。

- 为了减少封账号的风险，脚本里面采用的时间、滑动位置方法都是随机的。

- 经测试，基本上每天刷个1小时，后面的金币的量就微乎其微了。


### 使用脚本

打开Appium，链接，再打开脚本即可。

In [ ]:
from appium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time,random,datetime
from appium.webdriver.common.touch_action import TouchAction
from appium.webdriver.common.mobileby import MobileBy


# Appium 基本参数
PLATFORM = 'Android'
DEVICE_NAME = 'SM_C7000'
DRIVER_SERVER = 'http://localhost:4723/wd/hub'
TIMEOUT = 600
NORESET = 'True'
FLICK_STRAT_X = 300
FLICK_START_Y = 300
FLICK_DISTANCE = 700


# 趣头条操作元素
popup_ele = 'com.jifen.qukan:id/rv'
home_ele = 'com.jifen.qukan:id/jo'
coin_ele = 'com.jifen.qukan:id/vh'
mission_ele = 'com.jifen.qukan:id/js'
video_ele = 'com.jifen.qukan:id/jp'
content_ele = 'com.jifen.qukan:id/a43'
time_text_ele = 'com.jifen.qukan:id/a45'
svideo_ele = 'com.jifen.qukan:id/jq'
sz_menu_ele = '/hierarchy/android.widget.FrameLayout/android.widget.LinearLayout/android.widget.FrameLayout/android.widget.LinearLayout/android.widget.FrameLayout/android.widget.RelativeLayout/android.widget.FrameLayout[2]/android.widget.LinearLayout/android.widget.RelativeLayout/android.widget.RelativeLayout/android.widget.HorizontalScrollView/android.widget.LinearLayout/android.widget.TextView[3]'
articles_ele = 'com.jifen.qukan:id/a35'
return_ele = 'com.jifen.qukan:id/kp'

class Automation():
    
    # 初始化 Appium 基本参数
     def __init__(self, APP_PACKAGE, APP_ACTIVITY):
        self.desired_caps = {
            'platformName': PLATFORM,
            'deviceName': DEVICE_NAME,
            'appPackage': APP_PACKAGE,
            'appActivity': APP_ACTIVITY,
            'noReset': NORESET,
            'newCommandTimeout': TIMEOUT,
        }
        print('打开 appium 服务器...')
        print('配置 appium ...')
        self.driver = webdriver.Remote(DRIVER_SERVER, self.desired_caps)
        self.wait = WebDriverWait(self.driver, 30)
        self.size = self.driver.get_window_size()
    
    # 屏幕方法
    def swipeUp(self):
        '''向上滑动屏幕'''
        self.driver.swipe(self.size['width'] * random.uniform(0.09, 0.45),
                          self.size['height'] * random.uniform(0.65, 0.89),
                          self.size['width'] * random.uniform(0.55, 0.95),
                          self.size['height'] * random.uniform(0.11, 0.35), random.uniform(800, 1200))
        print('向上滑动屏幕')

    def swipeDown(self):
        '''向下滑动屏幕'''
        self.driver.swipe(self.size['width'] * random.uniform(0.09, 0.45),
                          self.size['height'] * random.uniform(0.11, 0.35),
                          self.size['width'] * random.uniform(0.55, 0.95),
                          self.size['height'] * random.uniform(0.65, 0.89), random.uniform(800, 1200))
        print('向下滑动屏幕')

    def swipeRight(self):
        '''向右滑动屏幕'''
        self.driver.swipe(self.size['width'] * random.uniform(0.01, 0.11),
                          self.size['height'] * random.uniform(0.75, 0.89),
                          self.size['width'] * random.uniform(0.25, 0.35),
                          self.size['height'] * random.uniform(0.75, 0.89), random.uniform(800, 1200))
        print('向右滑动屏幕')

class Qutoutiao(Automation):
    
    # 领金币
    def get_coins(self):
        try:
            pop_up = self.wait.until(EC.presence_of_element_located((By.ID, popup_ele)))
            pop_up.click()
            print('取消推送通知...')
        except:
            print('无推送通知弹出框...')
        home = self.wait.until(EC.presence_of_element_located((By.ID, home_ele)))
        home.click()
        print('点击底部菜单:头条...')
        try:
            coins = self.wait.until(EC.presence_of_element_located((By.ID, coin_ele)))
            time.sleep(2)
            if coins.get_attribute('text') == '领取':
                coins.click()
                print('领取金币')
                time.sleep(1)
            else:
                print('无金币可领取')
        except:
            pass
    
    # 每日签到
    def mission(self):
        mission = self.wait.until(EC.presence_of_element_located((By.ID, mission_ele)))
        mission.click()
        print('点击底部菜单:任务...')
        print('签到...')
        time.sleep(2)

    # 视频
    def video(self, interval):
        video = self.wait.until(EC.presence_of_element_located((By.ID, video_ele)))
        video.click()
        print('点击底部菜单:视频...')
        t = 1
        start = time.time()
        while True:
            content = self.driver.find_elements_by_id(content_ele)
            time_text = self.driver.find_elements_by_id(time_text_ele)
            print('视频数量', len(content))
            for i in range(len(content)):
                try:
                    print('第{}次进入循环.. 点击第{}视频'.format(t, i + 1))
                    time_span = time_text[i].get_attribute('text').split(':')
                    content[i].click()
                    time_cost = int(time_span[0]) * 60 + int(time_span[1])
                    time.sleep(random.randint(4, 10))
                    print('该视频时长:{}s'.format(time_cost))
                    time.sleep(time_cost)
                except:
                    print('此为广告...')
                    continue
            t += 1
            self.swipeUp()
            time.sleep(random.randint(5, 20))
            end = start + interval
            now = time.time()
            if now >= end:
                print('结束小视频循环')
                break
    
    # 小视频
    def smallvideo(self, interval):
        smallvideo = self.wait.until(EC.presence_of_element_located((By.ID, svideo_ele)))
        smallvideo.click()
        print('点击底部菜单:小视频...')
        start = time.time()
        while True:
            time.sleep(random.randint(5, 20))
            self.swipeUp()
            end = start + interval
            now = time.time()
            if now >= end:
                print('结束小视频循环')
                break
    
    # 阅读新闻
    def read_article(self, interval):
        try:
            pop_up = self.wait.until(EC.presence_of_element_located((By.ID, popup_ele)))
            pop_up.click()
            print('取消签到提醒...')
        except:
            print('无签到提醒弹出框...')
        home = self.wait.until(EC.presence_of_element_located((By.ID, home_ele)))
        home.click()
        print('点击底部菜单:头条...')
        sz_menu = self.wait.until(EC.presence_of_element_located((By.XPATH,sz_menu_ele)))
        sz_menu.click()
        print('点击顶部菜单:深圳...')
        start = time.time()
        while True:
            try:
                articles = self.wait.until(EC.presence_of_all_elements_located((By.ID, articles_ele)))
                print('新闻数量', len(articles))
                for article in articles:
                    print('进入新闻：{}'.format(article.get_attribute('text')))
                    article.click()
                    for i in range(random.randint(2, 6)):
                        time.sleep(random.randint(6, 10))
                        self.swipeUp()
                    for i in range(random.randint(2, 6)):
                        time.sleep(random.randint(6, 10))
                        self.swipeDown()
            except:
                pass
            finally:
                return_btn = self.wait.until(EC.presence_of_element_located((By.ID, return_ele)))
                return_btn.click()
                print('退出新闻进入菜单')

            for i in range(random.randint(2, 4)):
                time.sleep(random.randint(1, 3))
                self.swipeUp()
                print('菜单深圳')
            end = start + interval
            now = time.time()
            if now >= end:
                print('结束新闻循环')
                break
            else:
                print('未到结束时间')
    
    # 入口函数
    def main(self):
        print('打开 {} App ...'.format('趣头条'))
        self.mission()
        func_list = ['smallvideo', 'read_article', 'video']  # 新闻阅读，小视频，视频三个方法随机
        for i in range(12):
            self.get_coins()
            exp = 'self.{}({})'.format(random.choice(func_list), random.randint(500, 700))
            print(exp)
            eval(exp)
        print('App {} 今日任务完成！'.format('趣头条'))


qutoutiao_appPackage = 'com.jifen.qukan'
qutoutiao_appActivity = 'com.jifen.qkbase.main.MainActivity'

if __name__ == '__main__':
    qutoutiao = Qutoutiao(qutoutiao_appPackage, qutoutiao_appActivity)
    qutoutiao.main()